# NLP -  ATIVIDADE PRÁTICA  - CLASSIFICAÇÃO DE TEXTO USANDO MACHINE LEARNING

Nesta prática iremos classificar um texto a partir de algoritmos de classificação e implementar um Random Forest. Para resolução do problema de classificação, passaremos por algumas etapas, conforme discutido em nossos estudos.

## O que é classificação de texto?
A Classificação de Texto é um processo automatizado de classificação em categorias predefinidas. Podemos classificar e-mails em spam ou não spam, artigos de notícias em diferentes categorias, como política, mercado de ações, esportes, etc.

Isso pode ser feito com a ajuda de Processamento de Linguagem Natural e diferentes Algoritmos de Classificação como Naive Bayes, SVM e até Redes Neurais em Python.

Usaremos o conjunto de dados de reviews da Amazon que possui 10.000 linhas de dados de texto classificados em “Rótulo 1” e “Rótulo 2”. O conjunto de dados tem duas colunas “Texto” e “Rótulo”. Você pode baixar os dados em https://raw.githubusercontent.com/Gunjitbedi/Text-Classification/master/corpus.csv .

# Importar bibliotecas


In [8]:
import pandas as pd
import numpy as np
import nltk
from nltk.tokenize import word_tokenize
from nltk import pos_tag
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.preprocessing import LabelEncoder
from collections import defaultdict
from nltk.corpus import wordnet as wn
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import model_selection, naive_bayes, svm
from sklearn.ensemble import RandomForestClassifier

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

from sklearn.metrics import accuracy_score
""" 
PERGUNTA 1 
Insira os modulos do NLTK para fazer download
"""
### SEU CODIGO AQUI ###

nltk.download('punkt')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Julio\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Julio\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Julio\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Julio\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

## Definir sementes aleatórias

Isso é usado para reproduzir o mesmo resultado todas as vezes se o script for mantido consistente, caso contrário, cada execução produzirá resultados diferentes. A semente pode ser definida para qualquer número.

In [9]:
""" 
PERGUNTA 2 
A definição de sementes aleatórias pode ser definida pelo seguinte código
"""
### SEU CODIGO AQUI ###
np.random.seed(500)

## Adicionando o corpus

In [10]:
# Mount your drive and past the correct path for read ther corpus
'''
OBS: Você deve conectar o Notebook ao seu drive pessoal e indicar o caminho correto para acesso ao dataset corpus.csv
'''
### SEU CODIGO AQUI ###
Corpus = pd.read_csv(r"./corpus.csv",encoding='latin-1')

In [11]:
""" 
PERGUNTA 3
Qual o tipo de dados da variável Corpus criada?
"""
### SEU CODIGO AQUI ###
type(Corpus)

pandas.core.frame.DataFrame

In [12]:
Corpus.head()

,text,label
0,Stuning even for the non-gamer: This sound tr...,__label__2
1,The best soundtrack ever to anything.: I'm re...,__label__2
2,Amazing!: This soundtrack is my favorite musi...,__label__2
3,Excellent Soundtrack: I truly like this sound...,__label__2
4,"Remember, Pull Your Jaw Off The Floor After H...",__label__2


## Pre processamento
Transformar dados brutos em um formato compreensível para modelos de PLN. Os dados do mundo real geralmente são incompletos, inconsistentes e provavelmente contêm muitos erros. O pré-processamento de dados é um método comprovado de resolver esses problemas. Isso ajudará na obtenção de melhores resultados por meio dos algoritmos de classificação.

In [13]:
"""
PERGUNTA 4
Para remover linhas em brancos se houver, usamos o seguinte trecho de codigo:
"""
# Step - a: Remove blank rows if any.
### SEU CODIGO AQUI ###
Corpus['text'].dropna(inplace=True)

"""
PERGUNTA 5
Para passar todo o texto para letras minusculas, usamos o seguinte trecho de codigo:
"""
# Step - b : Change all the text to lower case. This is required as python interprets 'dog' and 'DOG' differently
### SEU CODIGO AQUI ###
Corpus['text'] = [entry.lower() for entry in Corpus['text']]

"""
PERGUNTA 6
Para quebrar o corpus em um conjunto de palavras, usamos o seguinte trecho de código:
"""
# Step - c : Tokenization : In this each entry in the corpus will be broken into set of words
### SEU CODIGO AQUI ###
Corpus['text'] = [word_tokenize(entry) for entry in Corpus['text']]

# Step - d : Remove Stop words, Non-Numeric and perfom Word Stemming/Lemmenting.
# WordNetLemmatizer requires Pos tags to understand if the word is noun or verb or adjective etc. By default it is set to Noun
"""
PERGUNTA 7
Para fazermos o mapa de taggeamento das palavras em Adjetivo, Verbo e Adverbio, usamos o seguinte trecho de código:
"""
### SEU CODIGO AQUI ###
tag_map = defaultdict(lambda : wn.NOUN)
tag_map['J'] = wn.ADJ
tag_map['V'] = wn.VERB
tag_map['R'] = wn.ADV


for index,entry in enumerate(Corpus['text']):
    # Declaring Empty List to store the words that follow the rules for this step
    Final_words = []
    # Initializing WordNetLemmatizer()
    """
    PERGUNTA 8
    Para iniciar o WordNet lemmatizer, usamos o seguinte trecho de código:
    """
    ### SEU CODIGO AQUI ###
    word_Lemmatized = WordNetLemmatizer()
    
    
    # pos_tag function below will provide the 'tag' i.e if the word is Noun(N) or Verb(V) or something else.
    for word, tag in pos_tag(entry):
        # Below condition is to check for Stop words and consider only alphabets
        if word not in stopwords.words('english') and word.isalpha():
            word_Final = word_Lemmatized.lemmatize(word,tag_map[tag[0]])
            Final_words.append(word_Final)
    # The final processed set of words for each iteration will be stored in 'text_final'
    Corpus.loc[index,'text_final'] = str(Final_words)

## Preparar o conjunto de treino e teste

In [34]:
"""
PERGUNTA 9
Para separar o conjunto entre treino e teste com 70% para treino e 30% para teste, usamos o seguinte trecho de código:
"""
Train_X, Test_X, Train_Y, Test_Y = model_selection.train_test_split(Corpus['text_final'], Corpus['label'], test_size=0.2)
### SEU CODIGO AQUI ###

## Codificação
Codificar rótulos (labels) na variável de destino — Isso é feito para transformar dados categóricos do tipo string no conjunto de dados em valores numéricos que o modelo pode entender.

In [35]:
"""
PERGUNTA 10
Para transformar dados categóricos do tipo string no conjunto de dados em valores numéricos que o modelo pode entender,
usamos o seguinte trecho de código:
"""
### SEU CODIGO AQUI ###
Encoder = LabelEncoder()
Train_Y = Encoder.fit_transform(Train_Y)
Test_Y = Encoder.fit_transform(Test_Y)

## Vetorização de palavras
É um processo geral de transformar uma coleção de documentos de texto em vetores de recursos numéricos. Existem muitos métodos para converter dados de texto em vetores que o modelo pode entender, mas de longe o método mais popular é chamado TF-IDF. Este é um acrônimo que significa “Frequência de Termo – Documento Inverso” Frequência que são os componentes das pontuações resultantes atribuídas a cada palavra.

> **Term Frequency:** resume a frequência com que uma determinada palavra aparece em um documento.

> **Inverse Document Frequency:** Isso reduz as palavras que aparecem muito nos documentos.

Sem entrar na matemática, TF-IDF são pontuações de frequência de palavras que tentam destacar palavras que são mais interessantes, por exemplo, frequentes em um documento, mas não em todos os documentos.

A sintaxe a seguir pode ser usada para ajustar primeiro o modelo TF-IDF em todo o corpus. Isso ajudará o TF-IDF a construir um vocabulário de palavras que aprendeu com os dados do corpus e atribuirá um número inteiro único a cada uma dessas palavras. Serão no máximo 5000 palavras/características únicas, pois definimos o parâmetro max_features=5000.

Finalmente vamos transformar Train_X e Test_X para Train_X_Tfidf vetorizado e Test_X_Tfidf . Estes agora conterão para cada linha uma lista de números inteiros exclusivos e sua importância associada conforme calculado pelo TF-IDF.

In [36]:
"""
PERGUNTA 10
Ao utilizar o TF-IDF, com o tamanho máximo do vocabulário definido em 5000, qual trecho de código devemos utilizar?
"""
### SEU CODIGO AQUI ###
Tfidf_vect = TfidfVectorizer(max_features=5000)
Tfidf_vect.fit(Corpus['text_final'])
Train_X_Tfidf = Tfidf_vect.transform(Train_X)
Test_X_Tfidf = Tfidf_vect.transform(Test_X)

Para ver o vocabulário aprendido com o Corpus

In [18]:
'''
PERGUNTA 11 
Para sabermos qual o vocabulário aprendido pelo Corpus, usamos usamos o seguinte trecho de código:
O que esse vocabulário representa e qual é o seu tipo?
'''
### SEU CODIGO AQUI ###
print(Tfidf_vect.vocabulary_)

{'stun': 4279, 'even': 1525, 'sound': 4131, 'track': 4559, 'beautiful': 385, 'paint': 3167, 'mind': 2838, 'well': 4865, 'would': 4952, 'recomend': 3605, 'people': 3236, 'hate': 2054, 'video': 4768, 'game': 1870, 'music': 2930, 'play': 3310, 'cross': 1024, 'ever': 1529, 'best': 427, 'back': 328, 'away': 318, 'crude': 1027, 'take': 4374, 'fresh': 1830, 'step': 4219, 'guitar': 2004, 'orchestra': 3104, 'impress': 2246, 'anyone': 203, 'care': 645, 'listen': 2617, 'soundtrack': 4132, 'anything': 204, 'read': 3571, 'lot': 2661, 'review': 3722, 'say': 3842, 'figure': 1699, 'write': 4958, 'disagree': 1231, 'bit': 449, 'ultimate': 4636, 'masterpiece': 2753, 'timeless': 4506, 'year': 4977, 'beauty': 387, 'simply': 4019, 'refuse': 3629, 'price': 3416, 'tag': 4373, 'pretty': 3411, 'must': 2936, 'go': 1932, 'buy': 604, 'cd': 676, 'much': 2920, 'money': 2877, 'one': 3082, 'feel': 1675, 'worth': 4948, 'every': 1531, 'penny': 3235, 'amaze': 153, 'favorite': 1663, 'time': 4505, 'hand': 2024, 'intense': 

Podemos imprimir diretamente os dados vetorizados para ver como fica

> **Saída:** — 1: Número da linha de 'Train_X_Tfidf', 2: Número inteiro único de cada palavra na primeira linha, 3: Pontuação calculada pelo TF-IDF Vectorizer



In [19]:
print(Train_X_Tfidf)

  (0, 4506)	0.37634188677099956
  (0, 4505)	0.1502086671688917
  (0, 3979)	0.35870975205557054
  (0, 3894)	0.25152943577361386
  (0, 3862)	0.2690840463105974
  (0, 3746)	0.3469774999759746
  (0, 3663)	0.28971770688512954
  (0, 3571)	0.29440491517773787
  (0, 2936)	0.22969709983777647
  (0, 1940)	0.13398240399394393
  (0, 1525)	0.17762585383071805
  (0, 521)	0.3210759641783664
  (0, 491)	0.1230432680090133
  (0, 240)	0.24487094004433968
  (1, 4692)	0.36974013511943044
  (1, 4077)	0.6167222431544791
  (1, 3441)	0.367922932130556
  (1, 2593)	0.3755181501193181
  (1, 1253)	0.3587203442870721
  (1, 604)	0.27907786873623097
  (2, 4739)	0.21254760778273238
  (2, 4628)	0.17350284094477353
  (2, 4466)	0.11900470145263356
  (2, 4200)	0.13517671323243532
  (2, 3854)	0.26661119879415307
  :	:
  (6998, 2522)	0.11515961575144278
  (6998, 2125)	0.13654425766350872
  (6998, 1972)	0.07125207506420554
  (6998, 1789)	0.22020146972839663
  (6998, 1753)	0.19941178219513117
  (6998, 1715)	0.1351314704611587

Dessa forma, os conjuntos de dados estão prontos para serem alimentados em diferentes algoritmos de classificação.

## Algoritmos de ML para prever o resultado
### Naive Bayes


In [20]:
# Classificador - Algoritmo - NB
# ajuste o conjunto de dados de treinamento no classificador NB 
Naive = naive_bayes.MultinomialNB() 
Naive.fit(Train_X_Tfidf,Train_Y)
# prever os rótulos no conjunto de dados de validação 
predictions_NB = Naive.predict(Test_X_Tfidf)
# Use a função precision_score para obter a precisão 
print("Naive Bayes Accuracy Score -> ",accuracy_score(predictions_NB, Test_Y)*100)

Naive Bayes Accuracy Score ->  83.26666666666667


### SVM

In [21]:
# Classificador - Algoritmo - SVM 
# ajusta o conjunto de dados de treinamento no classificador 
SVM = svm.SVC(C=1.0, kernel='linear', degree=3, gamma='auto') 
SVM.fit(Train_X_Tfidf,Train_Y)
# prever os rótulos no conjunto de dados de validação 
predictions_SVM = SVM.predict(Test_X_Tfidf)
# Use a função precision_score para obter a precisão 
print("SVM Accuracy Score -> ",accuracy_score(predictions_SVM, Test_Y)*100)

SVM Accuracy Score ->  84.56666666666666


### Random Forest

In [ ]:
"""
Com base na documentação do Scikilearn e dos algoritmos Naive Bayes e SVM apresentados em nossas aulas, codifique um classificador Random Forest 
(consulte a documentação do Scikit-learn e tome como exemplo os classificadores Naive Bayes e SVM implementados no Notebook) e responda as seguintes questões:

PERGUNTA 12
Considerando os valores de (n_estimators = 10, random_state = 0) e o conjunto de treino e
teste como 70/30, o Random Forest teve a sua acurácia prevista na faixa de qual porcentagem?

PERGUNTA 13
Considerando os valores de (n_estimators = 100, random_state = 0) e o conjunto de treino e teste como 80/20, 
o Random Forest, Naive Bayes e SVM, em relação a acurácia obtida, marque a alternativa correta...

PERGUNTA 14
Considerando os valores de (n_estimators = 100, random_state = 0) e o conjunto de treino e teste como 80/20 
em relação ao Random Forest, a seguinte afirmação está correta...

PERGUNTA 15
Pensando na perspectiva de melhoria dos modelos de Machine Learning, podemos avaliar o ajuste de hiper parâmetros, considerando as seguintes técnicas...


PARA SE PENSAR...
Como saber se o nosso modelo criado está generalizando de maneira adequada?

- A base possui um tamanho adequado?
- O classificador é adequado para o problema em questão?

"""

# Classificador - Algoritmo - RF
# Needed for the next step in model parameter tuning
Train_X, Test_X, Train_Y, Test_Y

# random forest test
# Instantiate classifier
### SEU CODIGO AQUI ###

# fit on training data
### SEU CODIGO AQUI ###

# prever os rótulos no conjunto de dados de validação 
### SEU CODIGO AQUI ###

# Use a função precision_score para obter a precisão 
### SEU CODIGO AQUI ###

# Seeing the metrics
#print("Accuracy on training set: {:.3f}".format(forest.score(Train_X_Tfidf,Train_Y)))
#print("Accuracy on test set: {:.3f}".format(forest.score(Test_X_Tfidf, Test_Y)))

In [22]:
'''PERGUNTA 12
Considerando os valores de (n_estimators = 10, random_state = 0) e o conjunto de treino e
teste como 70/30, o Random Forest teve a sua acurácia prevista na faixa de qual porcentagem?'''
# Classificador - Algoritmo - Rando Forest 
# ajusta o conjunto de dados de treinamento no classificador 
RFC = RandomForestClassifier(n_estimators= 10, random_state= 0) 
RFC.fit(Train_X_Tfidf,Train_Y)
# prever os rótulos no conjunto de dados de validação 
predictions_RFC = RFC.predict(Test_X_Tfidf)
# Use a função precision_score para obter a precisão 
print("RFC Accuracy Score -> ",accuracy_score(predictions_RFC, Test_Y)*100)

RFC Accuracy Score ->  76.5


In [37]:
'''PERGUNTA 13
Considerando os valores de (n_estimators = 100, random_state = 0) e o conjunto de treino e teste como 80/20, 
o Random Forest, Naive Bayes e SVM, em relação a acurácia obtida, marque a alternativa correta...'''

# Classificador - Algoritmo - Rando Forest 
# ajusta o conjunto de dados de treinamento no classificador 
RFC = RandomForestClassifier(n_estimators= 100, random_state= 0) 
RFC.fit(Train_X_Tfidf,Train_Y)
# prever os rótulos no conjunto de dados de validação 
predictions_RFC = RFC.predict(Test_X_Tfidf)
# Use a função precision_score para obter a precisão 
print("RFC Accuracy Score -> ",accuracy_score(predictions_RFC, Test_Y)*100)

RFC Accuracy Score ->  81.85


In [ ]:
'''PERGUNTA 14
Considerando os valores de (n_estimators = 100, random_state = 0) e o conjunto de treino e teste como 80/20 
em relação ao Random Forest, a seguinte afirmação está correta...
'''

# O RF possui uma boa acurácia, mas pode ser melhorado com técnicas de otimização

In [ ]:
'''PERGUNTA 15
Pensando na perspectiva de melhoria dos modelos de Machine Learning, 
podemos avaliar o ajuste de hiper parâmetros, considerando as seguintes técnicas...
'''
# Random Search, Grid Search, Cross Validation.
